# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [5]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [6]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)

from tensorflow import random
random.set_seed(2)

In [7]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [8]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.head()

Date
2018-01-02    14754.129883
2018-01-03    15156.620117
2018-01-04    15180.080078
2018-01-05    16954.779297
2018-01-06    17172.300781
Name: Close, dtype: float64

In [9]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [10]:
df['Close']= round(df['Close'], 1)
df.tail()

,fng_value,Close
2019-07-25,42,9882.4
2019-07-26,24,9847.5
2019-07-27,47,9478.3
2019-07-28,16,9531.8
2019-07-29,19,9529.9


In [11]:
df.isnull().sum()

 fng_value    0
Close         0
dtype: int64

In [12]:
def window_data(df, window, feature_col_number, target_col_number):
    """
    This function accepts the column number for the features (X) and the target (y).
    It chunks the data up with a rolling window of Xt - window to predict Xt.
    It returns two numpy arrays of X and y.
    """
    X = []
    y = []
    
    for rows in range(len(df) - window - 1):
        features = df.iloc[rows:(rows + window), feature_col_number]
        target = df.iloc[(rows + window), target_col_number]
        
        X.append(features)
        y.append(target)
    
    return np.array(X), np.array(y).reshape(-1, 1)

In [13]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window= 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_col_number = 1
target_col_number = 1

X, y = window_data(df, window, feature_col_number, target_col_number)

In [14]:
print (f"X sample values:\n{X[:5]} \n")
print (f"y sample values:\n{y[:5]}")

X sample values:
[[9114.7 8870.8 9251.3 8218.  6937.1 7701.2 7592.7 8260.7 8696.8 8569.3]
 [8870.8 9251.3 8218.  6937.1 7701.2 7592.7 8260.7 8696.8 8569.3 8084.6]
 [9251.3 8218.  6937.1 7701.2 7592.7 8260.7 8696.8 8569.3 8084.6 8911.3]
 [8218.  6937.1 7701.2 7592.7 8260.7 8696.8 8569.3 8084.6 8911.3 8544.7]
 [6937.1 7701.2 7592.7 8260.7 8696.8 8569.3 8084.6 8911.3 8544.7 9485.6]] 

y sample values:
[[ 8084.6]
 [ 8911.3]
 [ 8544.7]
 [ 9485.6]
 [10033.8]]


In [15]:
print (len (X))

532


In [16]:
# Use 70% of the data for training and the remainder for testing
split= int (0.7* len(X))

X_train= X[: split]
X_test= X[split: ]

y_train= y[: split]
y_test= y[split: ]

In [17]:
from sklearn.preprocessing import MinMaxScaler

# Use the MinMaxScaler to scale data between 0 and 1.
scaler= MinMaxScaler()

scaler.fit(X)
X_train_scaled= scaler.transform(X_train)
X_test_scaled= scaler.transform(X_test)

scaler.fit(y)
y_train_scaled= scaler.transform(y_train)
y_test_scaled= scaler.transform(y_test)

In [18]:
print (f"X_train_scaled sample values:\n {X_train_scaled[:2]}\n")
print (f"y_train_scaled sample values:\n {y_train_scaled[:2]}")

X_train_scaled sample values:
 [[0.60761507 0.58242087 0.62172548 0.51498843 0.38267499 0.46160441
  0.45039666 0.51939922 0.56444715 0.55127675]
 [0.58242087 0.62172548 0.51498843 0.38267499 0.46160441 0.45039666
  0.51939922 0.56444715 0.55127675 0.50120858]]

y_train_scaled sample values:
 [[0.50120858]
 [0.58660441]]


In [19]:
print (f"X_train_scaled shape: {X_train_scaled.shape}")
print (f"X_test_scaled shape: {X_test_scaled.shape}")
print (f"y_train_scaled shape: {y_train_scaled.shape}")
print (f"y_test_scaled shape: {y_test_scaled.shape}")

X_train_scaled shape: (372, 10)
X_test_scaled shape: (160, 10)
y_train_scaled shape: (372, 1)
y_test_scaled shape: (160, 1)


In [20]:
# Reshape the features for the model
X_train_reshaped= X_train_scaled.reshape((X_train_scaled.shape[0], X_train_scaled.shape[1], 1))

X_test_reshaped= X_test_scaled.reshape((X_test_scaled.shape[0], X_test_scaled.shape[1], 1))

In [21]:
print (f"X_train_reshaped sample values:\n{X_train_reshaped[:1]} \n")
print (f"X_test_reshaped sample values:\n{X_test_reshaped[:1]}")

X_train_reshaped sample values:
[[[0.60761507]
  [0.58242087]
  [0.62172548]
  [0.51498843]
  [0.38267499]
  [0.46160441]
  [0.45039666]
  [0.51939922]
  [0.56444715]
  [0.55127675]]] 

X_test_reshaped sample values:
[[[0.04675233]
  [0.039129  ]
  [0.03975911]
  [0.03857119]
  [0.03679448]
  [0.03729031]
  [0.03973845]
  [0.04528551]
  [0.04528551]
  [0.07025246]]]


In [22]:
print (f"X_train_reshaped shape: {X_train_reshaped.shape}")
print (f"X_test_reshaped shape: {X_test_reshaped.shape}")

X_train_reshaped shape: (372, 10, 1)
X_test_reshaped shape: (160, 10, 1)


---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

### Defining the LSTM RNN Model Structure
- The return sequences need to be set to True if you are adding additional LSTM layers, but you don't have to do this for the final layer. 
- The dropouts help prevent overfitting
- The input shape is the number of time steps and the number of indicators
- Batching inputs has a different input shape of Samples/TimeSteps/Features

In [36]:
# Define the LSTM RNN model.
model= Sequential()

number_units= 30
dropout_fraction= 0.2

# Layer 1
model.add (LSTM (units= number_units, return_sequences= True, input_shape= (X_train_reshaped.shape[1], 1)))
model.add (Dropout (dropout_fraction))

# Layer 2
model.add (LSTM (units= number_units, return_sequences= True))
model.add (Dropout (dropout_fraction))

# Layer 3
model.add (LSTM (units= number_units))
model.add (Dropout (dropout_fraction))

# Output layer
model.add (Dense (units= 1))

In [37]:
# Compile the model
model.compile (loss= 'mean_squared_error', optimizer= 'adam')

In [38]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 30)            3840      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [47]:
# Train the model
# Use at least 10 epochs, Do not shuffle the data
# Experiement with smaller batch size

model.fit (X_train_reshaped, y_train_scaled, epochs= 20, batch_size= 20, shuffle= False)

Epoch 1/20
19/19 [==============================] - 0s 15ms/step - loss: 0.0066
Epoch 2/20
19/19 [==============================] - 0s 13ms/step - loss: 0.0058
Epoch 3/20
19/19 [==============================] - 0s 15ms/step - loss: 0.0054
Epoch 4/20
19/19 [==============================] - 0s 15ms/step - loss: 0.0057
Epoch 5/20
19/19 [==============================] - 0s 14ms/step - loss: 0.0053
Epoch 6/20
19/19 [==============================] - 0s 15ms/step - loss: 0.0055
Epoch 7/20
19/19 [==============================] - 0s 15ms/step - loss: 0.0052
Epoch 8/20
19/19 [==============================] - 0s 14ms/step - loss: 0.0061
Epoch 9/20
19/19 [==============================] - 0s 14ms/step - loss: 0.0060
Epoch 10/20
19/19 [==============================] - 0s 15ms/step - loss: 0.0055
Epoch 11/20
19/19 [==============================] - 0s 14ms/step - loss: 0.0054
Epoch 12/20
19/19 [==============================] - 0s 15ms/step - loss: 0.0047
Epoch 13/20
19/19 [==================

In [48]:
# epocs = [1, 5, 10, 15]
# batch_size = [1, 5, 10, 15]
# module_res = []
# for x in epocs:
#     for y in batch_size:
#         model.fit (X_train_reshaped, y_train_scaled, epochs= x, batch_size= y, shuffle= False)
#         module_res.append({
#             "epochs": x,
#             "batch_size": y,
#             "score": score,
#             "rmse": rmse,
#         })

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [49]:
# Evaluate the model
model.evaluate (X_test_reshaped, y_test_scaled)

5/5 [==============================] - 0s 4ms/step - loss: 0.0078


0.007794261910021305

In [50]:
# Make some predictions
predicted= model.predict (X_test_reshaped)
predicted[:5]

array([[0.04006409],
       [0.04250174],
       [0.04637377],
       [0.05083588],
       [0.05554644]], dtype=float32)

In [51]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test_scaled.reshape(-1, 1))

In [52]:
# Create a DataFrame of Real and Predicted values
real_predicted = pd.DataFrame(
    {
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
    }, 
    index = df.index[-len(real_prices): ]
) 

real_predicted.tail()

,Real,Predicted
2019-07-25,9772.1,9614.197266
2019-07-26,9882.4,9582.246094
2019-07-27,9847.5,9523.398438
2019-07-28,9478.3,9450.321289
2019-07-29,9531.8,9348.173828


In [53]:
real_predicted.shape

(160, 2)

In [54]:
# Plot the real vs predicted values as a line chart
real_predicted.hvplot(title= 'Real vs predicted price of Bitcoin')

:NdOverlay   [Variable]
   :Curve   [index]   (value)